# Локальный Jupyter-пайплайн для пакетной обработки изображений

Этот ноутбук реализует устойчивый к повторным запускам инкрементальный пайплайн для:
- нормализации изображений в PNG,
- переименования новых исходников,
- сборки PDF по модулям,
- добавления кликабельного оглавления в PDF (OCR + PyMuPDF),
- сборки PPTX (2 изображения на слайд),
- полной пересборки нормализации.

Установка зависимостей:
```bash
pip install pillow natsort img2pdf pymupdf pandas pytesseract python-pptx tqdm
```

> Важно: Tesseract OCR должен быть установлен в Windows отдельно, а путь к `tesseract.exe` нужно указать в соответствующей ячейке OCR.


## Этап 1. Настройки и функции нормализации

**Что делает:** задаёт базовый конфиг и функции нормализации изображения (автообрезка фона по цвету угла, вписывание в холст, сохранение PNG), а также функции обхода исходников и вычисления целевых путей.

**Когда запускать:** в начале сессии и после любых изменений логики нормализации.

**Зависимости:** нет. Это базовая ячейка для этапов 2–7.


In [ ]:
from pathlib import Path
from PIL import Image
import csv
import io
import re
import time
import traceback
from datetime import datetime
from typing import List

# ===== Конфиг этапа 1 =====
SRC_ROOT = Path(r"C:\data\screenshots")
OUT_ROOT = SRC_ROOT / "_normalized"
TARGET_W = 1920
TARGET_H = 1080
TOLERANCE = 12
LOG_PATH = OUT_ROOT / "_normalize_log.csv"
PROBLEMS_PATH = OUT_ROOT / "_normalize_problems.txt"

VALID_EXTS = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp"}
OUT_ROOT.mkdir(parents=True, exist_ok=True)

def _ts() -> str:
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def _append_problem(msg: str) -> None:
    PROBLEMS_PATH.parent.mkdir(parents=True, exist_ok=True)
    with PROBLEMS_PATH.open("a", encoding="utf-8") as f:
        f.write(f"[{_ts()}] {msg}\n")

def autocrop_by_corner_bg(img: Image.Image, tol: int) -> Image.Image:
    img_rgba = img.convert("RGBA")
    px = img_rgba.load()
    w, h = img_rgba.size
    bg = px[0, 0]

    def is_bg(p):
        return all(abs(int(p[i]) - int(bg[i])) <= tol for i in range(3))

    left, top, right, bottom = w, h, -1, -1
    for y in range(h):
        for x in range(w):
            if not is_bg(px[x, y]):
                left = min(left, x)
                top = min(top, y)
                right = max(right, x)
                bottom = max(bottom, y)

    if right < left or bottom < top:
        return img_rgba

    return img_rgba.crop((left, top, right + 1, bottom + 1))

def fit_to_canvas(img: Image.Image, target_w: int, target_h: int, bg=(255, 255, 255, 0)) -> Image.Image:
    src = img.convert("RGBA")
    sw, sh = src.size
    if sw == 0 or sh == 0:
        raise ValueError("Пустое изображение")

    scale = min(target_w / sw, target_h / sh)
    nw, nh = max(1, int(sw * scale)), max(1, int(sh * scale))
    resized = src.resize((nw, nh), Image.Resampling.LANCZOS)

    canvas = Image.new("RGBA", (target_w, target_h), bg)
    ox = (target_w - nw) // 2
    oy = (target_h - nh) // 2
    canvas.paste(resized, (ox, oy), resized)
    return canvas

def list_source_files() -> List[Path]:
    files = [
        p for p in SRC_ROOT.rglob("*")
        if p.is_file() and p.suffix.lower() in VALID_EXTS and OUT_ROOT not in p.parents
    ]
    files.sort()
    return files

def dst_path_for(src_path: Path) -> Path:
    rel = src_path.relative_to(SRC_ROOT)
    return (OUT_ROOT / rel).with_suffix(".png")

def normalize_one(in_path: Path, out_path: Path) -> bool:
    out_path.parent.mkdir(parents=True, exist_ok=True)
    try:
        with Image.open(in_path) as im:
            cropped = autocrop_by_corner_bg(im, TOLERANCE)
            fitted = fit_to_canvas(cropped, TARGET_W, TARGET_H)
            fitted.save(out_path, format="PNG", optimize=True)
        return True
    except Exception as e:
        _append_problem(f"normalize_one error: src={in_path} dst={out_path} err={e}")
        return False

print(f"[ok] База инициализирована. SRC_ROOT={SRC_ROOT}")
print(f"[info] Выход нормализации: {OUT_ROOT}")


## Этап 2. Переименование только новых исходных файлов

**Что делает:** переименовывает только файлы, которые ещё не имеют имени формата `NNN.ext` внутри каждой папки, продолжая нумерацию от максимального номера.

**Когда запускать:** перед нормализацией, если хотите унифицировать имена новых файлов.

**Зависимости:** этап 1 (конфиг `SRC_ROOT`, `VALID_EXTS`).


In [ ]:
# ===== Конфиг этапа 2 =====
DRY_RUN = True

num_re = re.compile(r"^(\d{3,})$")

def rename_new_files(dry_run: bool = True):
    stats = {"folders": 0, "planned": 0, "renamed": 0, "skipped_numbered": 0, "errors": 0}
    plans = []

    for folder in sorted([p for p in SRC_ROOT.rglob("*") if p.is_dir() and OUT_ROOT not in p.parents and p != OUT_ROOT]):
        files = sorted([f for f in folder.iterdir() if f.is_file() and f.suffix.lower() in VALID_EXTS])
        if not files:
            continue

        stats["folders"] += 1
        max_num = 0
        pending = []
        for f in files:
            stem = f.stem
            m = num_re.match(stem)
            if m:
                max_num = max(max_num, int(m.group(1)))
                stats["skipped_numbered"] += 1
            else:
                pending.append(f)

        next_num = max_num + 1
        for src in pending:
            dst = src.with_name(f"{next_num:03d}{src.suffix.lower()}")
            while dst.exists():
                next_num += 1
                dst = src.with_name(f"{next_num:03d}{src.suffix.lower()}")
            plans.append((src, dst))
            next_num += 1

    stats["planned"] = len(plans)
    print(f"[info] План переименования: {len(plans)} файлов")
    for src, dst in plans:
        print(f"[build] {src} -> {dst.name}")

    if dry_run:
        print("[skip] DRY_RUN=True, файловая система не изменена")
    else:
        for src, dst in plans:
            try:
                src.rename(dst)
                stats["renamed"] += 1
                print(f"[ok] renamed: {src.name} -> {dst.name}")
            except Exception as e:
                stats["errors"] += 1
                _append_problem(f"rename error: {src} -> {dst}, err={e}")
                print(f"[error] {src} -> {dst}: {e}")

    print("[info] Summary rename:", stats)
    return stats

rename_stats = rename_new_files(dry_run=DRY_RUN)


## Этап 3. Инкрементальная нормализация (mtime)

**Что делает:** обрабатывает только новые/изменённые исходники, ведёт append-лог в CSV и append-лог проблем в TXT.

**Когда запускать:** после этапа 1, обычно после этапа 2 (если было переименование).

**Зависимости:** этап 1.


In [ ]:
# ===== Конфиг этапа 3 =====
FORCE_REBUILD = False

def needs_processing(src: Path, dst: Path) -> bool:
    if not dst.exists():
        return True
    return src.stat().st_mtime > dst.stat().st_mtime

def ensure_csv_header(path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    if not path.exists() or path.stat().st_size == 0:
        with path.open("w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["ts", "src", "dst", "status", "elapsed_sec", "error"])

def append_csv_row(path: Path, row: list):
    with path.open("a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(row)

def run_incremental_normalize(force_rebuild: bool = False):
    started = time.perf_counter()
    ensure_csv_header(LOG_PATH)

    processed = 0
    skipped = 0
    problems = 0

    for src in list_source_files():
        dst = dst_path_for(src)
        if (not force_rebuild) and (not needs_processing(src, dst)):
            skipped += 1
            print(f"[skip] up-to-date: {src}")
            continue

        t0 = time.perf_counter()
        ok = normalize_one(src, dst)
        dt = round(time.perf_counter() - t0, 4)
        if ok:
            processed += 1
            append_csv_row(LOG_PATH, [_ts(), str(src), str(dst), "ok", dt, ""])
            print(f"[ok] normalized: {src} -> {dst}")
        else:
            problems += 1
            append_csv_row(LOG_PATH, [_ts(), str(src), str(dst), "error", dt, "normalize failed"])
            print(f"[error] normalize failed: {src}")

    total = round(time.perf_counter() - started, 2)
    print(f"[info] Summary normalize: processed={processed}, skipped={skipped}, problems={problems}, time={total}s")

run_incremental_normalize(force_rebuild=FORCE_REBUILD)


## Этап 4. Сборка PDF по модулям из `_normalized`

**Что делает:** формирует PDF по подпапкам-модулям (`000__intro` идёт первой), с natural sort, инкрементальным пропуском актуальных файлов, и fallback `img2pdf -> PIL`.

**Когда запускать:** после нормализации (этап 3 или 7).

**Зависимости:** этап 1 (структура и пути нормализованных PNG).


In [ ]:
from natsort import natsorted
import img2pdf

# ===== Конфиг этапа 4 =====
PDF_OUT_ROOT = SRC_ROOT / "_pdf_modules"
INCREMENTAL = True
PDF_OUT_ROOT.mkdir(parents=True, exist_ok=True)

def module_dirs(root: Path):
    subs = [d for d in root.iterdir() if d.is_dir()] if root.exists() else []
    if not subs:
        return [("all", root)]

    def key(d: Path):
        return (0 if d.name.lower() == "000__intro" else 1, d.name.lower())

    ordered = sorted(subs, key=key)
    return [(d.name, d) for d in ordered]

def module_images(mod_path: Path):
    files = [p for p in mod_path.rglob("*.png") if p.is_file()]
    return natsorted(files, key=lambda x: str(x.relative_to(mod_path)).lower())

def _latest_mtime(paths):
    return max((p.stat().st_mtime for p in paths), default=0)

def build_pdf(images: list[Path], out_pdf: Path):
    out_pdf.parent.mkdir(parents=True, exist_ok=True)
    try:
        with out_pdf.open("wb") as f:
            f.write(img2pdf.convert([str(p) for p in images]))
        return "img2pdf"
    except Exception:
        pil_images = []
        try:
            for p in images:
                pil_images.append(Image.open(p).convert("RGB"))
            first, rest = pil_images[0], pil_images[1:]
            first.save(out_pdf, save_all=True, append_images=rest)
            return "PIL"
        finally:
            for im in pil_images:
                im.close()

def run_pdf_build(incremental: bool = True):
    for mod_name, mod_dir in module_dirs(OUT_ROOT):
        images = module_images(mod_dir)
        if not images:
            print(f"[skip] module={mod_name}: нет PNG")
            continue

        out_pdf = PDF_OUT_ROOT / f"{mod_name}.pdf"
        if incremental and out_pdf.exists() and out_pdf.stat().st_mtime >= _latest_mtime(images):
            print(f"[skip] module={mod_name}: PDF актуален")
            continue

        t0 = time.perf_counter()
        try:
            engine = build_pdf(images, out_pdf)
            dt = round(time.perf_counter() - t0, 2)
            print(f"[ok] module={mod_name} pages={len(images)} engine={engine} time={dt}s")
            print(f"[info] first={images[0]}")
            print(f"[info] last={images[-1]}")
        except Exception as e:
            _append_problem(f"pdf build error module={mod_name}: {e}\n{traceback.format_exc()}")
            print(f"[error] module={mod_name}: {e}")

run_pdf_build(incremental=INCREMENTAL)


## Этап 5. Кликабельное оглавление в PDF (OCR + PyMuPDF)

**Что делает:** ищет страницы оглавления через OCR (`pytesseract + pandas`), извлекает пункты с номерами страниц справа, и вставляет кликабельные ссылки `fitz.LINK_GOTO` в новый файл `*_toc.pdf`.

**Когда запускать:** после этапа 4, если нужно интерактивное оглавление в PDF.

**Зависимости:** этап 4 (готовые `*_pdf_modules/*.pdf`).


In [ ]:
import fitz
import pandas as pd
import pytesseract

# ===== Конфиг этапа 5 =====
TESSERACT_EXE = Path(r"C:\Program Files\Tesseract-OCR\tesseract.exe")
OCR_LANG = "rus+eng"
OCR_DPI = 220
PDF_GLOB = "*.pdf"
TOC_LOG_PATH = PDF_OUT_ROOT / "_toc_log.csv"
TOC_INCREMENTAL = True

if TESSERACT_EXE.exists():
    pytesseract.pytesseract.tesseract_cmd = str(TESSERACT_EXE)
    print(f"[ok] tesseract path set: {TESSERACT_EXE}")
else:
    print(f"[error] Не найден tesseract.exe: {TESSERACT_EXE}")

def render_page_for_ocr(doc: fitz.Document, page_index: int, dpi: int = 220) -> Image.Image:
    page = doc[page_index]
    mat = fitz.Matrix(dpi / 72, dpi / 72)
    pix = page.get_pixmap(matrix=mat, alpha=False)
    return Image.open(io.BytesIO(pix.tobytes("png"))).convert("RGB")

def _clean_ocr_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in ["left", "top", "width", "height", "conf"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df["text"] = df["text"].fillna("").astype(str).str.strip()
    df = df[(df["text"] != "") & (df["conf"].fillna(-1) >= 0)].copy()
    return df

def _extract_right_side_numbers(df: pd.DataFrame, page_w: int) -> pd.DataFrame:
    df2 = df.copy()
    df2["is_num"] = df2["text"].str.fullmatch(r"\d{1,4}")
    right_zone = df2["left"] > (page_w * 0.62)
    return df2[df2["is_num"] & right_zone].copy()

def _contains_toc_keyword(text: str) -> bool:
    t = text.lower().replace("ё", "е")
    keys = ["оглавлен", "содержан", "содержаиие", "огпавление", "coдepжaниe"]
    return any(k in t for k in keys)

def detect_toc_pages_and_entries(pdf_path: Path, max_scan_pages: int = 12):
    entries = []
    toc_pages = []
    doc = fitz.open(pdf_path)
    try:
        scan_pages = min(len(doc), max_scan_pages)
        for i in range(scan_pages):
            image = render_page_for_ocr(doc, i, dpi=OCR_DPI)
            data = pytesseract.image_to_data(image, lang=OCR_LANG, output_type=pytesseract.Output.DATAFRAME)
            df = _clean_ocr_df(data)
            if df.empty:
                continue

            full_text = " ".join(df["text"].tolist())
            num_df = _extract_right_side_numbers(df, image.width)

            soft_threshold = 1 if i < 3 else 2
            if _contains_toc_keyword(full_text) or len(num_df) >= soft_threshold:
                toc_pages.append(i)

                lines = {}
                for _, r in df.iterrows():
                    key = int(round(float(r["top"]) / 8.0) * 8)
                    lines.setdefault(key, []).append(r)

                for y, rows in sorted(lines.items(), key=lambda x: x[0]):
                    rows = sorted(rows, key=lambda r: float(r["left"]))
                    texts = [str(r["text"]) for r in rows]
                    if len(texts) < 2:
                        continue

                    last = texts[-1]
                    if not re.fullmatch(r"\d{1,4}", last):
                        continue

                    title = " ".join(texts[:-1]).strip(" .•·-—_")
                    if len(title) < 2:
                        continue

                    src_page_1based = int(last)
                    left_vals = [float(r["left"]) for r in rows]
                    top_vals = [float(r["top"]) for r in rows]
                    width_vals = [float(r["width"]) for r in rows]
                    height_vals = [float(r["height"]) for r in rows]
                    x0 = min(left_vals)
                    y0 = min(top_vals)
                    x1 = max(l + w for l, w in zip(left_vals, width_vals))
                    y1 = max(t + h for t, h in zip(top_vals, height_vals))

                    entries.append({
                        "toc_page": i,
                        "title": title,
                        "target_page_1based": src_page_1based,
                        "bbox": (x0, y0, x1, y1),
                    })

        uniq = []
        seen = set()
        for e in entries:
            key = (e["toc_page"], e["title"].lower(), e["target_page_1based"])
            if key not in seen:
                seen.add(key)
                uniq.append(e)

        return toc_pages, uniq
    finally:
        doc.close()

def add_toc_links(in_pdf: Path, out_pdf: Path, entries: list[dict]):
    doc = fitz.open(in_pdf)
    links_added = 0
    try:
        page_count = len(doc)
        for e in entries:
            src_i = int(e["toc_page"])
            dst_i = int(e["target_page_1based"]) - 1
            if src_i < 0 or src_i >= page_count or dst_i < 0 or dst_i >= page_count:
                continue

            x0, y0, x1, y1 = e["bbox"]
            rect = fitz.Rect(float(x0), float(y0), float(x1), float(y1))
            link = {
                "kind": fitz.LINK_GOTO,
                "from": rect,
                "page": dst_i,
                "zoom": 0,
            }
            doc[src_i].insert_link(link)
            links_added += 1

        out_pdf.parent.mkdir(parents=True, exist_ok=True)
        doc.save(out_pdf)
        return links_added
    finally:
        doc.close()

def ensure_toc_log_header(path: Path):
    if not path.exists() or path.stat().st_size == 0:
        with path.open("w", newline="", encoding="utf-8") as f:
            csv.writer(f).writerow(["ts", "pdf", "out", "status", "toc_pages", "entries", "links", "error"])

def run_toc_ocr_incremental():
    ensure_toc_log_header(TOC_LOG_PATH)
    done = skipped = no_toc = no_links = errors = 0

    pdfs = [p for p in PDF_OUT_ROOT.glob(PDF_GLOB) if p.is_file() and not p.name.endswith("_toc.pdf")]
    pdfs = natsorted(pdfs, key=lambda p: p.name.lower())

    for pdf in pdfs:
        out = pdf.with_name(f"{pdf.stem}_toc.pdf")
        if TOC_INCREMENTAL and out.exists() and out.stat().st_mtime >= pdf.stat().st_mtime:
            skipped += 1
            print(f"[skip] TOC актуален: {pdf.name}")
            continue

        try:
            toc_pages, entries = detect_toc_pages_and_entries(pdf)
            if not toc_pages:
                no_toc += 1
                with TOC_LOG_PATH.open("a", newline="", encoding="utf-8") as f:
                    csv.writer(f).writerow([_ts(), str(pdf), str(out), "no_toc", 0, 0, 0, ""])
                print(f"[skip] TOC не найден: {pdf.name}")
                continue

            links = add_toc_links(pdf, out, entries)
            if links == 0:
                no_links += 1
                status = "no_links"
                print(f"[skip] TOC найден, но ссылок нет: {pdf.name}")
            else:
                done += 1
                status = "done"
                print(f"[ok] TOC добавлен: {pdf.name} -> {out.name}, links={links}")

            with TOC_LOG_PATH.open("a", newline="", encoding="utf-8") as f:
                csv.writer(f).writerow([_ts(), str(pdf), str(out), status, len(toc_pages), len(entries), links, ""])

        except Exception as e:
            errors += 1
            err = f"{e}"
            _append_problem(f"toc error for {pdf}: {e}\n{traceback.format_exc()}")
            with TOC_LOG_PATH.open("a", newline="", encoding="utf-8") as f:
                csv.writer(f).writerow([_ts(), str(pdf), str(out), "error", 0, 0, 0, err])
            print(f"[error] TOC ошибка: {pdf.name}: {e}")

    print(f"[info] Summary TOC: done={done}, skipped={skipped}, no_toc={no_toc}, no_links={no_links}, errors={errors}")

run_toc_ocr_incremental()


## Этап 6. Сборка PPTX по модулям (2-up)

**Что делает:** создаёт презентации по модулям: титульный слайд и далее по 2 изображения на слайд. Есть инкрементальный пропуск по mtime.

**Когда запускать:** после нормализации (этап 3 или 7).

**Зависимости:** этап 1 (нормализованные PNG).


In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt

# ===== Конфиг этапа 6 =====
PPTX_OUT_ROOT = SRC_ROOT / "_pptx_modules"
PPTX_INCREMENTAL = True
PPTX_OUT_ROOT.mkdir(parents=True, exist_ok=True)

def build_pptx_2up(images: list[Path], out_pptx: Path, title: str):
    prs = Presentation()

    title_slide = prs.slides.add_slide(prs.slide_layouts[5])
    if title_slide.shapes.title is not None:
        title_slide.shapes.title.text = title

    slide_w = prs.slide_width.inches
    slide_h = prs.slide_height.inches
    margin = 0.3
    gap = 0.2

    box_w = slide_w - 2 * margin
    box_h = (slide_h - 2 * margin - gap) / 2

    for i in range(0, len(images), 2):
        slide = prs.slides.add_slide(prs.slide_layouts[6])
        chunk = images[i:i+2]
        for j, img_path in enumerate(chunk):
            top = margin + j * (box_h + gap)
            slide.shapes.add_picture(str(img_path), Inches(margin), Inches(top), width=Inches(box_w), height=Inches(box_h))

        note = slide.shapes.add_textbox(Inches(margin), Inches(slide_h - 0.35), Inches(box_w), Inches(0.25))
        tf = note.text_frame
        tf.text = f"{chunk[0].name}" + (f" | {chunk[1].name}" if len(chunk) > 1 else "")
        tf.paragraphs[0].font.size = Pt(10)

    out_pptx.parent.mkdir(parents=True, exist_ok=True)
    prs.save(out_pptx)

def run_pptx_build(incremental: bool = True):
    for mod_name, mod_dir in module_dirs(OUT_ROOT):
        images = module_images(mod_dir)
        if not images:
            print(f"[skip] module={mod_name}: нет PNG")
            continue

        out_pptx = PPTX_OUT_ROOT / f"{mod_name}.pptx"
        if incremental and out_pptx.exists() and out_pptx.stat().st_mtime >= _latest_mtime(images):
            print(f"[skip] module={mod_name}: PPTX актуален")
            continue

        t0 = time.perf_counter()
        try:
            build_pptx_2up(images, out_pptx, title=f"Модуль: {mod_name}")
            dt = round(time.perf_counter() - t0, 2)
            slides = 1 + (len(images) + 1) // 2
            print(f"[ok] module={mod_name} pages={len(images)} slides={slides} time={dt}s")
            print(f"[info] first={images[0]}")
            print(f"[info] last={images[-1]}")
        except Exception as e:
            _append_problem(f"pptx build error module={mod_name}: {e}\n{traceback.format_exc()}")
            print(f"[error] module={mod_name}: {e}")

run_pptx_build(incremental=PPTX_INCREMENTAL)


## Этап 7. Полная пересборка нормализации

**Что делает:** пересобирает все нормализованные PNG заново для всех исходников с прогресс-баром `tqdm`, полностью перезаписывая лог CSV и TXT проблем.

**Когда запускать:** когда нужно гарантированно пересоздать весь набор нормализованных данных (например, после изменения `TARGET_W/TARGET_H/TOLERANCE`).

**Зависимости:** этап 1.


In [ ]:
from tqdm import tqdm

# ===== Конфиг этапа 7 =====
FULL_REBUILD = False

def run_full_rebuild(enabled: bool = False):
    if not enabled:
        print("[skip] FULL_REBUILD=False, полная пересборка не запущена")
        return

    started = time.perf_counter()
    files = list_source_files()

    LOG_PATH.parent.mkdir(parents=True, exist_ok=True)
    with LOG_PATH.open("w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["ts", "src", "dst", "status", "elapsed_sec", "error"])
    PROBLEMS_PATH.write_text("", encoding="utf-8")

    processed = 0
    problems = 0

    for src in tqdm(files, desc="Full normalize", unit="img"):
        dst = dst_path_for(src)
        t0 = time.perf_counter()
        ok = normalize_one(src, dst)
        dt = round(time.perf_counter() - t0, 4)

        if ok:
            processed += 1
            append_csv_row(LOG_PATH, [_ts(), str(src), str(dst), "ok", dt, ""])
        else:
            problems += 1
            append_csv_row(LOG_PATH, [_ts(), str(src), str(dst), "error", dt, "normalize failed"])

    total = round(time.perf_counter() - started, 2)
    print(f"[info] Summary full rebuild: total={len(files)}, processed={processed}, problems={problems}, time={total}s")

run_full_rebuild(enabled=FULL_REBUILD)
